<a href="https://colab.research.google.com/github/ijusplab/implantacao-automatica/blob/master/Implantacao_LOAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação de Planilha de Execução Semi-Automática para B-87 e B-88

**INSTRUÇÕES**

1) Execute a célula `CARREGAR OFÍCIO GENÉRICO EM LOTE`. 

2) Assim que o código for inicializado e aparecer o botão de `upload`, carregue o arquivo `txt` do ofício genérico em lote. Você verá uma prévia do conteúdo extraído logo abaixo da célula, em forma de tabela.

3) Execute a célula `CARREGAR SENTENÇAS`. 

4) Quando aparecer o botão de `upload`, carregue os arquivos `pdf` das sentenças proferidas no lote. Novamente uma prévia do conteúdo extraído aparecerá numa tabela logo abaixo da célula.

5) Execute a célula `MESCLAR DADOS` para reunir os dados do ofício genérico em lote com os dados extraídos das sentenças. 

6) Execute a célula `BAIXAR ARQUIVO CSV` para baixar os dados extraídos, já devidamente mesclados, em formato `csv`.

In [0]:
#@title **CARREGAR OFÍCIO GENÉRICO EM LOTE** { vertical-output: true }

!pip install PyPDF2

from google.colab import files
import pandas as pd
import re, os, PyPDF2, requests, json
from IPython.display import display, HTML

##
## Para normalização dos nomes de colunas
##
class Utils:
  _instance = None

  def __init__(self):  
    self.__acentos = {
      'a': re.compile('[áàäãâÁÀÄÃÂ]'), 
      'e': re.compile('[éèëêÉÈËÊ]'), 
      'i': re.compile('[íìïîÍÌÏÎ]'),
      'o': re.compile('[óòöõôÓÒÖÕÔ]'),
      'u': re.compile('[úùüûÚÙÜÛ]'),
      'c': re.compile('[çÇ]')
    }

  @classmethod
  def instance(cls):
      if cls._instance is None:
          cls._instance = cls()
      return cls._instance

  def remover_acentuacao(self, s):
    for letra, padrao in self.__acentos.items():
      s = re.sub(padrao, letra, s)
    return s

  def normalizar(self, s):
    s = self.remover_acentuacao(s).upper().strip()
    s = re.sub(r'\d+_+', '', s)
    s = re.sub(r'\s+', '_', s)
    return s

  def ler_pdf(self, nome_arquivo):
    pdfFileObj = open(nome_arquivo, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    conteudo = []
    for num in range(pdfReader.numPages):
      pagina = pdfReader.getPage(num)
      try:
        texto = pagina.extractText()
      except:
        texto = ''
      conteudo.append(texto)
    return ''.join(conteudo)

##
## Instanciação
##
utils = Utils.instance()

##
## Extração do código IBGE dos municípios
##
class IBGE:
  _instance = None

  def __init__(self):  
    endpoint = 'https://servicodados.ibge.gov.br/api/v1/localidades/municipios'
    response = requests.get(endpoint)
    if response.status_code != 200:
      raise Exception(f'Erro ao tentar obter os dados do IBGE\sStatus Code: {response.status_code}')
    self.__dados_IBGE = response.json()

  @classmethod
  def instance(cls):
      if cls._instance is None:
          cls._instance = cls()
      return cls._instance

  def codigo_municipio(self, nome_municipio):
    for municipio in self.__dados_IBGE:
      if utils.normalizar(municipio['nome']) == utils.normalizar(nome_municipio):
        return municipio['id']
    return ''   

##
## Instanciação
##
ibge = IBGE.instance()

##
## Extração de dados de ofício genérico em lote
##
class OFGL:
  _instance = None

  def __init__(self):
    self.lote = ''
    self.data_frame = None

  @classmethod
  def instance(cls):
      if cls._instance is None:
          cls._instance = cls()
      return cls._instance

  def __transformar_cpf(self, val):
    s = str(val).replace('.', '').replace('-', '')
    return f'{s[:3]}.{s[3:6]}.{s[6:9]}-{s[9:11]}'

  def __transformar_data(self, val):
    pattern = re.compile('\d{1,2}[\/.\-]\d{1,2}[\/.\-]\d{2,4}')
    match = pattern.search(val)
    if match:
      return match.group(0).replace('.', '/').replace('-', '/')
    return val

  def __transformar_resultado(self, val):
    pattern = re.compile('PROCEDENTE EM PARTE|PROCEDENTE|ACORDO')
    match = pattern.search(val)
    if match:
      return match.group(0)
    return val

  def load(self):
    arquivos = files.upload()
    if len(arquivos) > 0:
      for nome, conteudo in arquivos.items():
        if re.match('^.+\.(txt)$', nome):
          df = pd.read_csv(nome, sep=';', encoding='latin1')
          df = df.fillna('')
          df.columns = map(utils.normalizar, df.columns)
          codigo_ibge = df['CIDADE_DO_POLO_ATIVO'].apply(ibge.codigo_municipio)
          df.insert(3, 'CODIGO_IBGE', codigo_ibge, True)
          df['CPF/CNPJ_POLO_ATIVO'] = df['CPF/CNPJ_POLO_ATIVO'].apply(self.__transformar_cpf)
          df['DATA_DO_TRANSITO'] = df['DATA_DO_TRANSITO'].apply(self.__transformar_data)
          df['RESULTADO_DA_SENTENCA'] = df['RESULTADO_DA_SENTENCA'].apply(self.__transformar_resultado)
          self.__lote = nome
          self.__data_frame = df
        else:
          print('>>>> ATENÇÃO')
          print('>>>> O arquivo ' + nome + ' não é um txt')

  def has_data_frame(self):
    return not self.__data_frame is None

  def get_name(self):
    if not self.has_data_frame():
      print('>>>> NÃO HÁ LOTE CARREGADO')
      return
    return self.__lote

  def get_data_frame(self):
    if not self.has_data_frame():
      print('>>>> NÃO HÁ LOTE CARREGADO')
      return
    return self.__data_frame

  def show_data_frame(self):
    if not self.has_data_frame():
      print('>>>> NÃO HÁ LOTE CARREGADO')
      return
    print('LOTE:')
    display(HTML(self.__data_frame.to_html()))

##
## Instanciação
##
ofgl = OFGL.instance()
ofgl.load()
ofgl.show_data_frame()

In [0]:
#@title **CARREGAR SENTENÇAS** { vertical-output: true }

##
## Extração de dados de sentenças
##
class Sentencas:
  def __init__(self):
    self.__padroes = {
        'PROCESSO': re.compile('PROCESSO:\s*?(\d{7}\-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4})', re.DOTALL),
        'ESPECIE': re.compile('ESP[ÉE]CIE DO NB:\s([À-ü]?.*)RMI', re.DOTALL),
        'DIB': re.compile('DIB:\s*?(\d{1,2}[/\-.]\d{1,2}[/\-.]\d{2,4})', re.DOTALL),
        'DIP': re.compile('DIP:\s*?(\d{1,2}[/\-.]\d{1,2}[/\-.]\d{2,4})', re.DOTALL)
    }
    self.__re_sumula = re.compile('\*{40,}([^*]+)\*{40,}', re.DOTALL)
    self.__sentencas = []
    self.__data_frame = None

  def __extrair_sumula(self, texto):
    match = self.__re_sumula.search(texto)
    if match:
      return match.group(1)
    return ''

  def __transformar_especie(self, val):
    return 88 if re.compile('IDOS[AO]|88').search(val.upper()) else 87

  def __create_data_frame(self):
    if len(self.__sentencas) > 0:
      colunas = self.__padroes.keys()
      dados = {}
      for rotulo, padrao in self.__padroes.items():
        coluna = []
        for sentenca in self.__sentencas:
          match = padrao.search(sentenca['conteudo'])
          if match:
            coluna.append(match.group(1))
          else:
            coluna.append('')
            print('>>>> Atributo "' + rotulo + '" não encontrado em ' + sentenca['nome'])
        dados[rotulo] = coluna
      df = pd.DataFrame(dados, columns=colunas)
      df = df.fillna('')
      df['ESPECIE'] = df['ESPECIE'].apply(self.__transformar_especie)
      self.__data_frame = df

  def load(self):
    arquivos = files.upload()
    if len(arquivos) > 0:
      for nome, conteudo in arquivos.items():
        if re.match('^.+\.(pdf)$', nome):
          conteudo = utils.ler_pdf(nome)
          self.__sentencas.append({
              'nome': nome,
              'conteudo': conteudo,
              'sumula': extrair_sumula(conteudo)
          })
        else:
          print('>>>> ATENÇÃO')
          print('>>>> O arquivo ' + nome + ' não é um pdf')
    self.__create_data_frame()

  def has_data_frame(self):
    return not self.__data_frame is None

  def get_data_frame(self):
    if not self.has_data_frame():
      print('>>>> NÃO HÁ SENTENÇAS CARREGADAS')
      return
    return self.__data_frame

  def show_data_frame(self):
    if not self.has_data_frame():
      print('>>>> NÃO HÁ SENTENÇAS CARREGADAS')
      return
    print('SENTENÇAS:')
    display(HTML(self.__data_frame.to_html()))

##
## Instanciação
##
sentencas = Sentencas()
sentencas.load()
sentencas.show_data_frame()

In [0]:
#@title **MESCLAR DADOS** { vertical-output: true }

##
## Para criar e baixar csv único
##
class CsvBuilder:
  def __init__(self):
    self.__data_frame = None
    self.__name = ''

  def load_data_frame(self, df, column):
    if self.__data_frame is None:
      self.__data_frame = df
    else:
      self.__data_frame = self.__data_frame.merge(df, on=column) 

  def set_name(self, name):
    self.__name = name

  def show_data_frame(self):
    if self.__data_frame is None:
      print('>>>> NÃO HÁ DADOS CARREGADOS')
      return
    if len(self.__name) == 0:
      print('>>>> NOME DO ARQUIVO NÃO FORNECIDO')
      return
    print(f'DADOS MESCLADOS ({self.__name}):')
    display(HTML(self.__data_frame.to_html()))

  def download(self):
    if self.__data_frame is None:
      print('>>>> NÃO HÁ DADOS CARREGADOS')
      return
    with open(f'{self.__name}.csv', 'w', encoding='latin1') as file:
      file.write(self.__data_frame.to_csv())
    files.download(f'{self.__name}.csv')

##
## Instanciação
##
builder = CsvBuilder()
builder.load_data_frame(ofgl.get_data_frame(), 'PROCESSO')
builder.load_data_frame(sentencas.get_data_frame(), 'PROCESSO')
builder.set_name(ofgl.get_name())
builder.show_data_frame()


In [0]:
#@title **BAIXAR ARQUIVO `CSV`**

builder.download()